In [14]:
!pip install ua_datasets
!pip install pyLDAvis

In [15]:
from ua_datasets import NewsClassificationDataset
train_data = NewsClassificationDataset(root = 'data/', split = 'train', return_tags = True)

In [153]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def read_text(name_txt):
  with open(name_txt, 'r') as file:
    text = file.readlines()
  return ' '.join(text)

In [112]:
train_data = pd.read_csv('data/train.csv')
train_data = train_data[:1000]

In [113]:
train_x = train_data['text']
train_y = train_data['target']

print(train_x.head(), '\n')
print(train_y.head())

0    Головний тренер солігорського «Шахтаря» Юрій В...
1    Про це на своїй сторінці у Facebook написав пр...
2    Про це повідомляється в доповіді некомерційної...
3     Легенда НБА Шакіл О’Ніл продав свій маєток у ...
4     Засновник фінансової піраміди B2B Jewelry Мик...
Name: text, dtype: object 

0     спорт
1    новини
2    новини
3     спорт
4    бізнес
Name: target, dtype: object


In [114]:
train_y.unique()

array(['спорт', 'новини', 'бізнес', 'політика', 'технології'],
      dtype=object)

Tokenization

In [115]:
def tokenize(data):
  tokens = []
  for line in data:
    tokens.append(nltk.word_tokenize(line.lower()))
  return tokens

train_x = tokenize(train_x)

Remove stop-words

In [142]:
def remove_punctuation_and_digits(data):
  punctuation = [',', '.', '/', '\\', '|', '\'', '\'\'', '\"', '«', '»', '-', '–', '—', '%', '`', ';', '``', '(', ')', '!', '?', ':', '’', '@', '#', '$', '+', 
                 '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'] 
  tokens_without_punct = []
  for token_sentence in data:
    tok = [token for token in token_sentence if token not in punctuation]
    tokens_without_punct.append(tok)
  return tokens_without_punct

train_x = remove_punctuation_and_digits(train_x)

In [143]:
def remove_stopwords(data):
  uk_stop_words = set(read_text('stopwords-uk.txt').split())
  tokens_without_sw = []
  for token_sentence in data:
    tok = [token for token in token_sentence if token not in uk_stop_words]
    tokens_without_sw.append(tok)
  return tokens_without_sw

train_x = remove_stopwords(train_x)

Build the bigram and trigram models

In [144]:
bigram = gensim.models.Phrases(train_x, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[train_x], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[train_x[0]]])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_bigrams = make_bigrams(train_x)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['головний_тренер', 'солігорського', 'шахтаря', 'юрій', 'вернидуб', 'післяматчевій', 'прес-конференції', 'пояснив', 'чому', 'підопічні', 'програли', 'команді', 'слуцьк', '1:2', 'такого', 'початку', "пам'ятаю", 'своєму', 'житті', 'то', 'є.', 'виправдовуватися', 'буду', 'команда', 'тій', 'грі', 'смолевичами', 'завдає', '21', 'удару', '24', '25', 'ударів', '15', 'ворота', 'забити', 'можемо', 'знаю', 'значить', 'вміємо', 'забивати', 'пенальті', 'можемо', 'реалізувати', 'забивай', '40', 'хвилин', 'часу', 'могло', 'по-іншому', 'маємо', 'маємо', 'заявці', 'сачівка', 'травму', 'отримав', 'грі', 'смолевичами', 'нього', 'міжреберна', 'невралгія', 'міг', 'вийти', 'шіндагорідзе', 'проблема', 'хрестоподібною', "зв'язкою", 'вирішиться', 'знаю', 'треба', 'робити', 'повторне', 'мрт', 'зрозумів', 'нього', 'така', 'проблема', 'то', 'треба', 'їхати', 'туди', 'робили', 'закрито', 'проблема', 'вистачило', 'обороні', 'першому', 'голі', 'просто', 'йшла', 'подача', 'удари', 'навпаки', 'нагнітали', 'моменти', 

Corpus and Dictionary

In [145]:
id2word = corpora.Dictionary(data_words_bigrams)

texts = data_words_bigrams

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 3), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1)]]


In [146]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('15', 1),
  ('1:2', 1),
  ('21', 1),
  ('24', 1),
  ('25', 1),
  ('30–35', 1),
  ('40', 1),
  ('буду', 1),
  ('вернидуб', 1),
  ('вийти', 1),
  ('вилазка', 1),
  ('виправдовуватися', 1),
  ('вирішиться', 1),
  ('вистачило', 1),
  ('вміємо', 1),
  ('ворота', 1),
  ('воріт', 1),
  ('говорити', 1),
  ('гол', 1),
  ('головний_тренер', 1),
  ('голі', 1),
  ('грі', 2),
  ('доводили', 1),
  ('же', 1),
  ('житті', 1),
  ('забивай', 1),
  ('забивати', 1),
  ('забити', 1),
  ('завдає', 1),
  ('завершення', 1),
  ('закрито', 1),
  ('заявці', 1),
  ("зв'язкою", 1),
  ('значить', 1),
  ('знаю', 2),
  ('зрозумів', 1),
  ('йшла', 2),
  ('команда', 1),
  ('команді', 1),
  ('логічного', 1),
  ('маємо', 2),
  ('метрів', 1),
  ('могло', 1),
  ('можемо', 2),
  ('моменти', 1),
  ('мрт', 1),
  ('міг', 1),
  ('міжреберна', 1),
  ('навпаки', 1),
  ('нагнітали', 1),
  ('невралгія', 1),
  ('нього', 2),
  ('ніхто', 1),
  ('обороні', 1),
  ('отримав', 1),
  ("пам'ятаю", 1),
  ('пенальті', 1),
  ('перекладина',

Build LDA Gensim model

In [147]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [148]:
lda_model.print_topics()

[(0,
  '0.004*"команди" + 0.003*"матчі" + 0.003*"динамо" + 0.003*"матч" + 0.003*"україни" + 0.002*"команда" + 0.002*"матчу" + 0.002*"проти" + 0.002*"клубу" + 0.002*"став"'),
 (1,
  '0.003*"україни" + 0.003*"росії" + 0.003*"заявив" + 0.003*"партії" + 0.003*"сказав" + 0.003*"підбирань" + 0.003*"президента" + 0.002*"повідомив" + 0.002*"рф" + 0.002*"криму"'),
 (2,
  '0.007*"україни" + 0.005*"україні" + 0.003*"року" + 0.003*"сша" + 0.002*"україна" + 0.002*"заявив" + 0.002*"єс" + 0.002*"повідомляє" + 0.002*"рішення" + 0.002*"компанії"'),
 (3,
  '0.002*"років" + 0.002*"землі" + 0.002*"фото" + 0.002*"spacex" + 0.002*"вчені" + 0.002*"знайшли" + 0.001*"лікарні" + 0.001*"14" + 0.001*"10" + 0.001*"the"'),
 (4,
  '0.003*"час" + 0.002*"більше" + 0.002*"багато" + 0.002*"наприклад" + 0.002*"компанії" + 0.002*"просто" + 0.002*"потрібно" + 0.002*"україні" + 0.002*"будуть" + 0.001*"то"')]

In [149]:
perplexity = lda_model.log_perplexity(corpus)
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [150]:
print('\nPerplexity: ', perplexity)
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.16784936094187

Coherence Score:  0.43417218307897637


In [151]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.167926 -0.143332       1        1  34.167804
4     -0.124237  0.175282       2        1  26.778228
0      0.116930  0.056475       3        1  15.553853
1      0.097120 -0.104657       4        1  14.572706
3      0.078112  0.016233       5        1   8.927409, topic_info=         Term        Freq       Total Category  logprob  loglift
195   україни  433.000000  433.000000  Default  30.0000  30.0000
606   україні  299.000000  299.000000  Default  29.0000  29.0000
2605  команди   82.000000   82.000000  Default  28.0000  28.0000
1347    матчі   71.000000   71.000000  Default  27.0000  27.0000
1332    росії  111.000000  111.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
2251    відео   12.920020   44.026264   Topic5  -6.8614   1.1900
414        10   15.561577  121.642055   Topic5  -6.6754   0.3598
936        км   10.006081   21.613287   Topic5  -7.1170   1.6459
1160      сша   13.830888  189.835309   Topic5  -6.7933  -0.2032
311      році   10.099751  110.484628   Topic5  -7.1077   0.0237

[306 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
414        1  0.304171    10
414        2  0.213742    10
414        3  0.123313    10
414        4  0.238404    10
414        5  0.131533    10
...      ...       ...   ...
1070       4  0.047007    єс
32194      1  0.983689   єіб
860        1  0.357999  інші
860        2  0.592550  інші
860        3  0.049379  інші

[462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 2, 4])

Build LDA Mallet model

In [163]:
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     
  !java -version       
install_java()

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [167]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2022-05-19 11:17:20--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-05-19 11:17:20--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  61.9MB/s    in 0.2s    

2022-05-19 11:17:21 (61.9 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mallet-2.0.8/bin/classifier2info  
replace mallet-2.0.8/bin/csv2classify? [y]e

In [169]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'

mallet_path = '/content/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [170]:
pprint(ldamallet.show_topics(formatted=False))

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('команди', 0.004380182862002977),
   ('10', 0.003869870295556028),
   ('о', 0.003742292153944291),
   ('матчі', 0.003657240059536466),
   ('матч', 0.00318945354029343),
   ('став', 0.00318945354029343),
   ('динамо', 0.003104401445885605),
   ('команда', 0.0029342972570699554),
   ('12', 0.0027641930682543057),
   ('11', 0.002679140973846481)]),
 (1,
  [('компанії', 0.007406362418000988),
   ('року', 0.005254990477534034),
   ('сша', 0.004584891020667278),
   ('компанія', 0.004232207096000564),
   ('україні', 0.004196938703533893),
   ('млн', 0.0038795231713338505),
   ('ринку', 0.003632644424067151),
   ('більше', 0.003562107639133808),
   ('понад', 0.0034210340692671227),
   ('млрд', 0.0031388869295337516)]),
 (2,
  [('україні', 0.006000468329235452),
   ('час', 0.005473597939351364),
   ('україни', 0.0038929867696990985),
   ('багато', 0.00380517503805175),
   ('немає', 0.0037173633064044023),
   ('єс', 0.0036880927291886197),
   ('сьогодні', 0.003541739843109706),
   ('пот